# Table of Contents
* [Current Infrastructure](#Current-Infrastructure)
	* [Single Server Hosting one JupyterHub Instance](#Single-Server-Hosting-one-JupyterHub-Instance)
* [New Infrastructure](#New-Infrastructure)
	* [Multiple Servers controlled through [Docker Swarm](https://docs.docker.com/engine/swarm/) Serving Multiple JupyterHub Instances](#Multiple-Servers-controlled-through-[Docker-Swarm]%28https://docs.docker.com/engine/swarm/%29-Serving-Multiple-JupyterHub-Instances)


# Current Infrastructure

## Single Server Hosting one JupyterHub Instance

* Authentication through GitHub openauth
    * Designed for educating non-UofU students as part of R25 DeCART Summer School
* 256 Unique Users over the past ~17 months
* Mark Keller (Eccles) system architect and administrator

<img
src="https://www.docker.com/sites/default/files/Package%20software.png" alt="Docker Containers" width="768">

# New Infrastructure

## Multiple Servers controlled through [Docker Swarm](https://docs.docker.com/engine/swarm/) Serving Multiple JupyterHub Instances

* Authentication with GitHUB or University of Utah ID

<img
src="./EcclesLearningEnvironment.png" alt="Learning Environment Schematic" width="768">


<img
src="./ClassDiagram.png" alt="Learning Environment Schematic" width="768">

```Yaml
version: '2'
services:
  notebook:
    build: ./jupyter
    ports:
      - "8989:8888"
    environment:
      - PASSWORD=bmi6950
      - USE_HTTPS=yes
    volumes:
      - /Users/brian/Documents/UUDS4HealthData/Data:/home/jovyan/DATA:ro
      - /Users/brian/Code/:/home/jovyan/work
    links:
      - mysolr
      - mynlp
      - mymongo
      - mysql
  mysolr:
    image: solr:latest
    hostname: mysolr
    domainname: localdomain
    ports:
      - "8983:8983"
  mynlp:
    image: motiz88/corenlp
    ports:
      - "9000:9000"
  mysql:
    image: mysql:latest
    environment:
      - MYSQL_ROOT_PASSWORD=mimic2-root-pw
    volumes:
      - /Users/brian/Bdrive/6950mysql-tmp:/data
  mymongo:
    image: mongo:latest
  mynginx:
    build: ./web-server
    ports:
        - "8080:80"
  myssh:
    build: ./ssh-server
    environment:
        - AUTH_MECHANISM=simpleAuth
        - AUTH_USER=jovyan
        - AUTH_PASSWORD=jovyan
        - HTTP_ENABLED=false
    ports:
        - "2212:22"
  myorient:
      image: orientdb:latest
      ports:
          - 2480:2480
          - 2424:2424
      environment:
          - ORIENTDB_ROOT_PASSWORD=root
```

```ruby
FROM jupyter/datascience-notebook:11be019e4079
MAINTAINER chapmanbe <brian.chapman@utah.edu>
USER root
# for physionet instructions see
# https://www.physionet.org/physiotools/wfdb-linux-quick-start.shtml
# dependencies for spell nbextensions (including spell check) and other requirements

RUN apt-get update && apt-get upgrade -y && apt-get install -y \
      tree \
      graphviz\ 
      less \
      openssh-client \
      gcc \
      libcurl4-openssl-dev \
      libexpat1-dev \
      libnetcdff5 \
      locales-all \
      libxrender-dev \
      libfontconfig-dev \
      libffi-dev \
      rsync \
      mongodb-clients \
      libmariadb-client-lgpl-dev \
  && rm -rf /var/lib/apt/lists/*


# Install physiosnet software
WORKDIR /opt/wfdb
RUN wget https://www.physionet.org/physiotools/wfdb.tar.gz && \
    tar xvfz wfdb.tar.gz 

WORKDIR /opt/wfdb/wfdb-10.5.24
RUN ./configure && \
    make install 
WORKDIR  /opt 
RUN rm -r /opt/wfdb 

WORKDIR /home/jovyan

# Set default editor to vim
RUN update-alternatives --set editor /usr/bin/vim.basic

USER jovyan

RUN conda install -c damianavila82 rise -y

RUN git clone https://github.com/Calysto/notebook-extensions.git
WORKDIR /home/jovyan/notebook-extensions
RUN jupyter nbextension install calysto --user && \
    jupyter nbextension enable calysto/document-tools/main && \
    jupyter nbextension enable calysto/cell-tools/main && \
    jupyter nbextension enable calysto/spell-check/main && \
    jupyter nbextension enable calysto/publish/main && \
    jupyter nbextension enable calysto/submit/main && \
    jupyter nbextension enable calysto/annotate/main 

RUN conda update conda -y && conda install -y \
    mpld3 \
    pip \
    pymysql \
    postgresql \
    gensim \
    biopython \
    pytables \
    scikit-image \
    pymongo \
    spacy 

RUN conda install -y  -c  https://conda.anaconda.org/simpleitk SimpleITK 
RUN conda install -y  -c conda-forge nbstripout
RUN pip install pydicom \
    textblob \
    ipythonblocks \
    pydotplus \
    nbstripout \
    pycontextnlp \
    radnlp \
    fhirclient \
    pivottablejs \
    girder_client \
    nose \
    pyorient

RUN R -e 'install.packages(c("tidyverse","tidytext","icd","RMySQL","DBI","dbplyr","SuperLearner","caret","tensorflow","ggdendro","tree"), dependencies = TRUE, repos = "https://cran.cnr.berkeley.edu/" )'

WORKDIR /home/jovyan
ADD decart_welcome.ipynb /home/jovyan/work/decart_welcome.ipynb
ADD media /home/jovyan/work/media
ADD dbmi_welcome.ipynb /home/jovyan/work/dbmi_welcome.ipynb
RUN conda install -y -c conda-forge nbgrader
# RUN nbstripout --install
CMD ["start-notebook.sh"]
```